In [63]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np



In [64]:
# تحميل البيانات
ratings = pd.read_csv(r"D:\task 4 data\ratings.csv")
movies = pd.read_csv(r"D:\task 4 data\movies.csv")

# ندمج بيانات التقييمات مع أسماء الأفلام
ratings = ratings.merge(movies, on='movieId')
ratings.head()


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [65]:
# نكوّن جدول المستخدمين × الأفلام
user_movie_matrix = ratings.pivot_table(index='userId', columns='title', values='rating')

# نملأ القيم الفارغة بصفر
user_movie_matrix_filled = user_movie_matrix.fillna(0)
user_movie_matrix_filled.head()



title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
# نحسب التشابه باستخدام Cosine Similarity
user_similarity = cosine_similarity(user_movie_matrix_filled)

# نحولها إلى DataFrame للمستخدمين
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix_filled.index, columns=user_movie_matrix_filled.index)


In [67]:
def get_user_recommendations(user_id, num_recommendations=10):
    # تشابه هذا المستخدم مع كل المستخدمين
    sim_scores = user_similarity_df[user_id]

    # نرتبهم حسب الأعلى تشابهًا (ونستبعد نفسه)
    similar_users = sim_scores.sort_values(ascending=False)[1:11]

    # ناخد التقييمات من المستخدمين المشابهين
    similar_users_ratings = user_movie_matrix.loc[similar_users.index]

    # نحسب متوسط التقييمات لكل فيلم
    weighted_ratings = similar_users_ratings.T.dot(similar_users) / similar_users.sum()

    # نشيل الأفلام اللي المستخدم شافها بالفعل
    user_rated_movies = user_movie_matrix.loc[user_id][user_movie_matrix.loc[user_id].notna()].index
    recommendations = weighted_ratings.drop(labels=user_rated_movies)

    # نرجع أفضل N توصية
    return recommendations.sort_values(ascending=False).head(num_recommendations)


In [68]:
# مثال: توصية للمستخدم رقم 1
print(get_user_recommendations(1))


title
Terminator 2: Judgment Day (1991)          4.195979
Hunt for Red October, The (1990)           3.957980
2001: A Space Odyssey (1968)               3.855366
Die Hard (1988)                            3.805436
Breakfast Club, The (1985)                 3.574258
'71 (2014)                                      NaN
'Hellboy': The Seeds of Creation (2004)         NaN
'Round Midnight (1986)                          NaN
'Salem's Lot (2004)                             NaN
'Til There Was You (1997)                       NaN
dtype: float64


In [69]:
# Load the ratings and movies datasets
ratings = pd.read_csv(r"D:\task 4 data\ratings.csv")
movies = pd.read_csv(r"D:\task 4 data\movies.csv")

# Merge to get movie titles with ratings
data = pd.merge(ratings, movies, on="movieId")

# Show sample
data.head()


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [70]:
# Create pivot table: users × movies with rating as values
user_movie_matrix = data.pivot_table(index='userId', columns='title', values='rating')

# Fill missing ratings with 0 (or you can use NaN depending on the method)
user_movie_matrix_filled = user_movie_matrix.fillna(0)


In [71]:
# Compute similarity matrix between users
user_similarity = cosine_similarity(user_movie_matrix_filled)

# Convert to DataFrame for readability
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)


In [72]:
def get_user_recommendations(user_id, num_recommendations=5):
    # Similarity scores of the target user with all other users
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:]

    # Weighted average rating from similar users
    weighted_ratings = pd.Series(dtype=np.float64)

    for other_user, similarity in similar_users.items():
        # Get that user's ratings
        other_user_ratings = user_movie_matrix.loc[other_user]

        # Weight the ratings by similarity
        weighted = other_user_ratings * similarity

        # Add to weighted_ratings
        weighted_ratings = weighted_ratings.add(weighted, fill_value=0)

    # Normalize by total similarity
    total_similarity = similar_users.sum()
    recommendations = weighted_ratings / total_similarity

    # Filter out already rated movies by the user
    user_seen_movies = user_movie_matrix.loc[user_id]
    recommendations = recommendations[user_seen_movies.isna()]

    # Return top N recommendations
    return recommendations.sort_values(ascending=False).head(num_recommendations)


In [73]:
# Try recommending movies for user with ID 10
recommended_movies = get_user_recommendations(user_id=10, num_recommendations=5)
print("🎬 Recommended movies for user 10:\n")
print(recommended_movies)


🎬 Recommended movies for user 10:

title
Shawshank Redemption, The (1994)                         2.775963
Star Wars: Episode IV - A New Hope (1977)                2.119340
Silence of the Lambs, The (1991)                         2.068510
Star Wars: Episode V - The Empire Strikes Back (1980)    1.892184
Inception (2010)                                         1.806329
dtype: float64


In [74]:
# تأكد إن الملفات مقروءة
ratings = pd.read_csv(r"D:\task 4 data\ratings.csv")
movies = pd.read_csv(r"D:\task 4 data\movies.csv")

# ✅ Step 1: ندمج الجدولين
merged_df = pd.merge(ratings, movies[['movieId', 'title']], on='movieId')

# ✅ Step 2: نعمل Pivot من الجدول المدموج (مش ratings)
user_movie_matrix = merged_df.pivot_table(index='userId', columns='title', values='rating')

# ✅ Step 3: نملأ القيم الفارغة بصفر
user_movie_matrix_filled = user_movie_matrix.fillna(0)

# ✅ عرض شكل المصفوفة
user_movie_matrix_filled.head()


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:

# Calculate cosine similarity between users
user_similarity = cosine_similarity(user_movie_matrix_filled)

# Convert to DataFrame for better visualization
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix_filled.index, columns=user_movie_matrix_filled.index)

user_similarity_df.head()


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.027283,0.059720,0.194395,0.129080,0.128152,0.158744,0.136968,0.064263,0.016875,...,0.080554,0.164455,0.221486,0.070669,0.153625,0.164191,0.269389,0.291097,0.093572,0.145321
2,0.027283,1.000000,0.000000,0.003726,0.016614,0.025333,0.027585,0.027257,0.000000,0.067445,...,0.202671,0.016866,0.011997,0.000000,0.000000,0.028429,0.012948,0.046211,0.027565,0.102427
3,0.059720,0.000000,1.000000,0.002251,0.005020,0.003936,0.000000,0.004941,0.000000,0.000000,...,0.005048,0.004892,0.024992,0.000000,0.010694,0.012993,0.019247,0.021128,0.000000,0.032119
4,0.194395,0.003726,0.002251,1.000000,0.128659,0.088491,0.115120,0.062969,0.011361,0.031163,...,0.085938,0.128273,0.307973,0.052985,0.084584,0.200395,0.131746,0.149858,0.032198,0.107683
5,0.129080,0.016614,0.005020,0.128659,1.000000,0.300349,0.108342,0.429075,0.000000,0.030611,...,0.068048,0.418747,0.110148,0.258773,0.148758,0.106435,0.152866,0.135535,0.261232,0.060792


In [76]:
def recommend_movies_user_based(user_id, num_recommendations=5):
    # Get similar users
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).drop(user_id)
    
    # Get movies rated by similar users
    weighted_ratings = pd.Series()
    
    for other_user, similarity in similar_users.items():
        other_user_ratings = user_movie_matrix_filled.loc[other_user]
        weighted_ratings = weighted_ratings.add(other_user_ratings * similarity, fill_value=0)
    
    # Normalize by sum of similarities
    weighted_ratings = weighted_ratings / similar_users.sum()

    # Remove movies already rated by the target user
    user_rated_movies = user_movie_matrix_filled.loc[user_id]
    unrated_movies = user_rated_movies[user_rated_movies == 0]
    
    recommendations = weighted_ratings[unrated_movies.index].sort_values(ascending=False).head(num_recommendations)
    
    return recommendations


In [77]:
# Try for user 1
recommend_movies_user_based(user_id=1, num_recommendations=10)


title
Shawshank Redemption, The (1994)                             2.654726
Terminator 2: Judgment Day (1991)                            2.087328
Godfather, The (1972)                                        1.859551
Sixth Sense, The (1999)                                      1.663571
Lord of the Rings: The Fellowship of the Ring, The (2001)    1.624831
Apollo 13 (1995)                                             1.585818
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)                    1.583816
Lord of the Rings: The Return of the King, The (2003)        1.502247
Lord of the Rings: The Two Towers, The (2002)                  1.4833
Aladdin (1992)                                               1.477036
dtype: object

In [78]:

user_similarity_df.iloc[0].sort_values(ascending=False).head(10)

userId
1      1.000000
266    0.357408
313    0.351562
368    0.345127
57     0.345034
91     0.334727
469    0.330664
39     0.329782
288    0.329700
452    0.328048
Name: 1, dtype: float64